In [21]:
from bokeh.plotting import figure, output_notebook, show
import math
from bokeh.models.widgets import Select
from bokeh.models import CustomJS, ColumnDataSource
from bokeh.layouts import column
import itertools
from bokeh.layouts import gridplot
from re import match

#self-defined module
import data_parser

output_notebook()
#import excel datasets, and convert them into dataframe
age_marriage_ds_df = data_parser.parse_excel_data('./data/indicator_age_of_marriage.xlsx', 'Data')
breast_female_mortality_ds_df = data_parser.parse_excel_data('./data/indicator_breast_female_mortality.xlsx', 'Data')

#Extract years and countries from the dataframe.
years = data_parser.extract_column_names(age_marriage_ds_df)
countries = data_parser.extract_row_names(age_marriage_ds_df)

#Extract years and countries from breast female mortality dataframe
breast_mortality_years = data_parser.extract_column_names(breast_female_mortality_ds_df)
breast_mortality_years.pop(0)
breast_mortality_countries = data_parser.extract_column_by_indice(breast_female_mortality_ds_df, 0)

#Convert dataframe into json format
data_json_list = data_parser.convert_df2json(age_marriage_ds_df, countries, years, "ages")

#ignore the first column of the dataset
tmp_breast_mortality_df = breast_female_mortality_ds_df.iloc[:, 1:]
breast_mortality_data_json_list = data_parser.convert_df2json(tmp_breast_mortality_df, breast_mortality_countries, breast_mortality_years, "mortality")
#print(breast_mortality_data_json_list)

#Initialize the plot
p = figure(plot_width=400, plot_height=400)
p.title.text = 'Age at 1st marriage (women) among different countries'
p.xaxis.axis_label = "Year"
p.yaxis.axis_label = "Age"

# Initialize plot1 for breast_female_mortality
p1 = figure(plot_width=400, plot_height=400)
p1.title.text = 'Breast cancer, deaths per 100,000 women'
p1.xaxis.axis_label = "Year"
p1.yaxis.axis_label = "Mortality number per 100,000 women"


#Move the data into datasource, which can be passed into customJS functions.
x = data_json_list[0]["years"]
y = data_json_list[0]["ages"]
source = ColumnDataSource(data=dict(x=x, y=y))
p.circle('x', 'y', source = source, size=10, alpha=0.5)
countries_source = ColumnDataSource(data=dict(countries=[x.lower().strip() for x in countries], data_json=data_json_list))

x1 = breast_mortality_data_json_list[0]["years"]
y1 = breast_mortality_data_json_list[0]["mortality"]
source1 = ColumnDataSource(data=dict(x=x1, y=y1))
p1.square('x', 'y', source = source1, size=10, alpha=0.5)
breast_mortality_countries_source = ColumnDataSource(data=dict(countries=[x.lower().strip() for x in breast_mortality_countries], data_json=breast_mortality_data_json_list))

def callback(source=source, countries=countries_source, source1=source1, countries1=breast_mortality_countries_source, window=None):
    f = cb_obj.value
    ci = countries.data['countries'].index(f.lower().strip())
    #print("c1: " + ci)
    ci1 = countries1.data['countries'].index(f.lower().strip())
    #print("ci1: " + ci1)
    source.data['x'] = countries.data['data_json'][ci]["years"] 
    source.data['y'] = countries.data['data_json'][ci]["ages"]
    if ci1 >= 0:
        source1.data['x'] = countries1.data['data_json'][ci1]["years"] 
        source1.data['y'] = countries1.data['data_json'][ci1]["mortality"]
    else:
        source1.data['x'] = []
        source1.data['y'] = []
    source.change.emit()
    source1.change.emit()

select = Select(title="Countries:", value=countries[0], options=countries, callback=CustomJS.from_py_func(callback))
grid = gridplot([[p, p1]])
layout = column(select, grid)


show(layout)



Loading BokehJS ...